# Deep Q-learning - Floating Maze
##### Max Brynolf (max.brynolf@hotmail.com)
The following code trains a deep reinforcement learning model to play a maze game coded in Java. The game has to be launched prior to running the code in this document.

### Import Packages

In [ ]:
# Basic packages
import numpy as np
import math
import time
import matplotlib.pyplot as plt
from IPython.display import clear_output
import random
from collections import deque

# Connection to Java
from py4j.java_gateway import JavaGateway

# Neural networks
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import copy

### Connect to the Java Application

The mainProcess-variable will represent the main class `Maze.java`. Before running this code, launch `Maze.java` with `TrainingMode.RL`.

In [ ]:
gateway = JavaGateway()
mainProcess = gateway.entry_point # mainProcess is the Maze class
mainProcess.startWindow(True)

### Action-Value Neural Network
The action-value function $Q(s_t, a_t)$, approximating $\mathbb{E}_{\pi}[G_t | s_t, a_t]$, is approximated by a neural network. The network structure is defined below. Note that the network outputs all possible actions $f(s) = [Q(s, a_1), Q(s, a_2), \dots , Q(s, a_n)]$, such that $Q(s, a_i) = f_i (s)$.

In [ ]:
class ActionValueNetwork(nn.Module):
    
    def __init__(self, screen_size, res, state_size, num_of_actions):
        super(ActionValueNetwork, self).__init__()

        conv_output_size = 32
        self.conv_layers = nn.ModuleList()
        self.conv_layers.append(nn.Conv2d(state_size, 16, kernel_size = 9, stride = 4, padding = 4, bias = True))
        self.conv_layers.append(nn.Conv2d(16, conv_output_size, kernel_size = 5, stride = 2, padding = 2, bias = True))
        
        # Calculate the input dimension to the FC-layer
        output_dim = round(screen_size / res)
        for conv_layer in self.conv_layers: #for i in range(len(self.conv_layers)):
            K = conv_layer.kernel_size[0]
            P = conv_layer.padding[0]
            S = conv_layer.stride[0]
            output_dim = math.floor((output_dim - K + 2*P)/S) + 1
        self.fc = nn.Linear(output_dim**2 * conv_output_size, num_of_actions)
    
    def forward(self, x):
        y = x
        for res_layer in self.conv_layers:
            y = F.relu(res_layer(y))
        y = torch.flatten(y, start_dim = 1)
        y = self.fc(y)
        return y

### Actions, States and Rewards
The following function converts the raw screen data sent from the Java program to a PyTorch tensor with the correct shape. The Java method `ScreenData.getAllPixels` provides a `byte[]`-array in the format:
$$
[K_{11}, K_{12}, K_{13}, \dots , K_{1M}, K_{21}, K_{22}, \dots , K_{MN}]
$$
where $K_{ij}$ is the grayscale decimal values for the pixel at position $(i, j)$. The method `ScreenData.getAllPixels` also includes a `resolution` parameter, which downsamples each dimension of the pixel data by a factor of `resolution`, before being converted to the vector above. For example, having a screen size of $300$ and setting `resolution = 2` returns pixel data of size $150 \cdot 150$. The output-tensor returned by the function has the same structure as the array above, but reshaped into a PyTorch tensor.

In [ ]:
def create_tensor_from_screen_data(data):
    arr = np.frombuffer(data, dtype=np.uint8).astype(np.float32) / 255
    tens = torch.from_numpy(arr)
    screen_dim = math.floor(math.sqrt(len(data))) # this holds for width = height, which is assumed throughout
    return tens.view(1, screen_dim, screen_dim)

The following functions allow the agent to take actions in the game and return the relevant data.
- `make_action(a, r, s, d)`: The agent takes action `a` for `d - 1` frames and stops the action for `1` frame. This makes sure that only one key is pressed at the same time. In return, it receives a `byte[]`-array of the form:
$$
[R, T, K_1, K_2, K_3, \dots , K_{n}]
$$
where $[K_i]$ contains the grayscale pixel data, with each screen dimension scaled by a factor of `1/r`, $R$ is the reward and $T$ is a variable indicating whether the episode has terminated or not. In order to calculate the reward from the current game score, the old score must be included in `s`. This makes $R$ the change in game score, i.e. the difference between the current game score and the game score at the last iteration, hence $R_t = s_t - s_{t - 1}$. The actions `a` correspond to different key presses along with a release key action:
$$
a \in \mathcal{A} = \{\leftarrow, \rightarrow, \uparrow, \downarrow, \varnothing \}
$$
Note that `d` cannot be lower than 2, since 1 frame is reserved for releasing the key. This behavior can of course be modified, but since a human can beat the game with way less frame-perfect control, using a single frame to only release keys can easily be justified.
- `make_action_realtime(a, r, t, d)`: Calls `make_action(a, r, 0, d)` with a delay `t` added such that subsequent calls to the function gives a viewer-friendly episode. This function can be called when testing the model. The old score is set to 0 since this doesn't matter when the model is not being trained.
- `reset()`: Resets the world.

In [ ]:
def make_action(action_type, screen_res, old_score = 0, frame_skip = 2):
    if (type(action_type) is np.int64):
        action_type = action_type.item()
    data = ""
    states = []
    
    if action_type == 4: # key released
        for i in range(frame_skip):
            data = mainProcess.stepWindowTraining(screen_res)
            states.append(data[2:])
            if data[1] == 1: # if episode terminates within frame skip
                break
    else: #key pressed
        for i in range(frame_skip):
            data = mainProcess.stepWindowTraining(action_type, i < frame_skip - 1, screen_res)
            states.append(data[2:])
            if data[1] == 1: # if episode terminates within frame skip
                break # if it breaks here, the new frames will not be used anyways, so it does not matter
    
    total_score = data[0]
    reward = total_score - old_score
    done = data[1]
    return states, reward, done, total_score

In [ ]:
def make_action_realtime(action_type, screen_res, interval, frame_skip = 1):
    states, _, done, _ = make_action(action_type, screen_res, 0, frame_skip)
    time.sleep(interval)
    return states, done

In [ ]:
def reset():
    mainProcess.reset()

### Deep Q-learning

To train the model, a set of parameters are needed. Their descriptions are included as comments to the right of each parameter. Some elaborated comments are needed for a few parameters, included below:

- `screen_dim`: Throughout the code, the window is always assumed to be a square. If this is to be changed, make sure to modify the code accordingly.
- `screen_res`: Note that increasing this will speed up the communication between Java and Python, but will reduce the resolution of the image seen by the neural network.
- `state_size`: For each step, `frame_skip` frames are appended to the current state. The state holds `state_size` frames and removes the oldest frames when it is full. Hence, if `frame_skip` isn't equal to `state_size`, it is possible that some frames will not be used, even if the code still works.

In [ ]:
# Environment parameters
screen_dim = mainProcess.getConstant(2) # screen dimension of the game
num_of_actions = 5; # number of possible actions you can take in the game
screen_res = 4 # defines how much the game is scaled down
timestep_interval = mainProcess.getConstant(4) # the timestep interval to be used during testing

# Step parameters
total_episodes = 10000 # number of episodes
target_nn_update_steps = 10000 # number of steps until the target network weights are updated
main_nn_update_steps = 100 # number of steps until the main network weights are updated
state_size = 4 # the size of the agent state, equivalent to the number of frames that are sent into the network
frame_skip = 4 # the number of frames before the state is updated, during which the same action is executed
max_steps = 1000 # the maximum number of steps before the episode is automatically terminated

# Exploration-exploitation trade-off
fixed_eps_episodes = 1000 # episodes at the start of training, in which exploration is set to eps_fixed
eps_fixed = 1 # the fixed value of eps, held for fixed_eps_episodes at the start of training
eps_max = 1 # maximum value of the exploration constant
eps_min = 0.1 # minimum value of the exploration constant
eps_decay_rate = 1.5 / (total_episodes - fixed_eps_episodes) # decay rate for the exploration constant
discount_factor = 0.99 # factor that specifies how important immediate reward is as opposed to long-term reward

# Other training parameters
replay_mem_min = 250 # the minimum size of the replay memory, for the neural network to be trained
replay_mem_length = 10000 # the size of the replay memory
adam_lr = 1e-3 # the learning rate used by the Adam optimizer within the gradient descent algorithm
batch_size = 64 # the batch size used when training the neural network
device = torch.device("cpu") # the device used during training
graph_update_frequency = 50 # number of episodes until the graphs are updated

# Print time-dependent parameters
episode_list = np.arange(total_episodes)
eps_list = [eps_fixed] * fixed_eps_episodes
eps_list += (eps_min + (eps_max - eps_min) * np.exp(-eps_decay_rate * (episode_list[fixed_eps_episodes:] - fixed_eps_episodes))).tolist()
plt.plot(episode_list, eps_list)
plt.title("Time-dependent parameters")
plt.xlabel("Episode")
plt.ylabel("Exploration constant")
plt.show();

The following function fits the neural network to the data in the replay memory. Given a memory `mem`, it randomly selects a batch of `batch_size` samples and updates the networks weights accordingly. The basis for the update is the Bellman Optimality Equation, relating the value of the optimal state-action-value function in the current state to its value in the next:
$$
Q^*(s, a) = r(s, a) + \gamma \sum_{s'} p(s' | s, a) \max_{a' \in \mathcal{A}} Q^*(s', a')
$$
Considering a transition from $s, a$ to $s'$, and letting the expectation $\sum_{s'} p(s' | s, a) \max_{a' \in \mathcal{A}} Q^*(s', a')$ be replaced with the sample $ \max_{a' \in \mathcal{A}} Q^*(s', a')$, we can use $r(s, a) + \gamma \max_{a' \in \mathcal{A}} Q(s', a')$ as a target when training our network. Note that $Q$ is bootstrapped in place of $Q^*$, since we cannot know the real $Q^*$. This can be shown to converge towards $Q^*$, under the right circumstances. In other words, the idea is to fit the neural network with respect to the one-step TD error $\delta$, defined as:
$$
\delta = Q(s, a) - \left(r(s, a) + \gamma \max_{a' \in \mathcal{A}} Q(s', a')\right)
$$
Since deep Q-learning is employed, $Q$ is approximated by $Q_\mathbf{w}$, where $\mathbf{w}$ is a vector with the neural network weights. The weights are updated in the following steps:
1. A batch is sampled from the replay memory, and the states $S_t$ along with their successive states $S_{t+1}$ are extracted.
2. The current approximation $Q(s_t, a_t)$ is calculated for each transition in the batch
3. The current approximation of the Q-values at $S_{t+1}$ are calculated using the target network, for each transition in the batch.
4. The target values Q-values $Y_t$ are calculated according to the Bellman equation $Y_t = r(s_t, a_t) + \gamma \max_a Q(s_{t+1}, a)$
5. The network weights are updated with respect to the loss between $Q(s_t, a_t)$ and $r(s_t, a_t) + \gamma \max_a Q(s_{t+1}, a)$. This is done using Huber loss, defined by:
$$
J(\delta) = \begin{cases}
\frac{1}{2}\delta^2, & \text{for } \left| \delta \right| \leq 1\\
\delta - \frac{1}{2} & \text{for } \left| \delta \right| > 1
\end{cases}$$
6. The loss is calculated and added to `loss_data`.

In [ ]:
def train(mem, main_nn, target_nn, optimizer, batch_size, device, loss_data):
    
    # Assure that replay memory is large enough
    if (len(mem) < batch_size):
        return
    
    # Sample a batch and extract states
    mini_batch = random.sample(mem, batch_size)
    states = np.array([transition[0] for transition in mini_batch])
    new_states = np.array([transition[3] for transition in mini_batch])
    
    # Calculate Q of current states and of the next states
    state_vals = main_nn(torch.from_numpy(states).to(device))
    next_state_vals = target_nn(torch.from_numpy(new_states).to(device)).detach().cpu().squeeze().numpy()
    
    # Create vectors with states and targets, based on the Bellman equation
    X = []
    Y = []
    for index, (S, action, reward, S_new, done) in enumerate(mini_batch):
        if done == 0:
            target = reward + discount_factor * np.max(next_state_vals[index])
        else:
            target = reward
        X.append(state_vals[index][action].reshape(1))
        Y.append(target)
    
    # Update the network weights with gradient descent
    X = torch.cat(X).to(device)
    Y = torch.Tensor(Y).to(device = device, dtype = torch.float32)
    loss_function = nn.SmoothL1Loss()
    optimizer.zero_grad()
    loss = loss_function(X, Y)
    loss.backward()
    optimizer.step()
    
    # Append loss data
    loss_data.append(loss.item())

The following functions are defined below:
- `routine(target_nn, rewards)`: Consists of the main training loop. It updates the user during the training process, while training the network `main_nn` and filling the `rewards`-list with rewards.
- `test_model(model, eps, episodes)`: Simulates `episodes` episodes in real-time using model `model`, with constant exploration set to `eps`.

In [ ]:
def routine(main_nn, rewards):
    try:
        
        # Initialize neural networks
        target_nn = copy.deepcopy(main_nn)
        main_nn.to(device)
        target_nn.to(device)
        optimizer = optim.Adam(main_nn.parameters(), lr = adam_lr)
        
        # Initialize plot
        ma_lengths = [100, 1000] # moving average lengths
        x = []
        y = []
        loss_data = [[],[]]
        
        # Reset environment
        reset()
        episode = 0
        mem = deque(maxlen = replay_mem_length) # replay memory
        target_update_counter = 0 # keeps track of when to update the target network
        main_update_counter = 0 # keeps track of when to update the main network

        # Loop through episodes
        for _ in range(total_episodes):
            
            # Reset or update episode-parameters
            done = 0
            score = 0
            state = deque(maxlen = state_size)
            used_steps = 0
            episode_reward = 0
            if episode < fixed_eps_episodes:
                eps = eps_fixed
            else:
                eps = eps_min + (eps_max - eps_min) * np.exp(-eps_decay_rate * (episode - fixed_eps_episodes))
            starting_frame = mainProcess.getPixelData(screen_res)
            for i in range(state_size):
                state.append(create_tensor_from_screen_data(starting_frame)) # starting state
            
            # Update graphs
            if (episode % graph_update_frequency == 0):
                update_graph(rewards, ma_lengths, loss_data)
                print("Episode: {0} out of {1} ({2}% finished)".format(episode, total_episodes, 100 * episode/total_episodes))
                print("Eps: {0}".format(eps))
                print("Replay memory size: {0}/{1} ({2}%)".format(len(mem), replay_mem_length, 100 * len(mem)/replay_mem_length))
            
            # Loop through steps
            while done == 0:
                
                # Reset or update parameters
                target_update_counter += 1
                main_update_counter += 1
                action = 0
                used_steps += 1
                
                # Decide whether to explore or exploit, and which action to take
                if np.random.uniform(0, 1) < eps:
                    action = np.random.randint(0, num_of_actions - 1)
                else:
                    input_tensor = torch.cat(tuple(state)).unsqueeze(0).to(device)
                    action = np.argmax(main_nn(input_tensor).detach().cpu().squeeze().numpy())
                
                # Take action and update state
                frames, reward, done, score = make_action(action, screen_res, score, frame_skip)
                new_state = state.copy()
                for frame in frames:
                    new_state.append(create_tensor_from_screen_data(frame))
                if (used_steps > max_steps):
                    reset()
                    done = 1
                
                # Update replay memory
                if len(state) == state_size:
                    numpy_state = torch.cat(tuple(state)).detach().numpy()
                    numpy_new_state = torch.cat(tuple(new_state)).detach().numpy()
                    mem.append([numpy_state, action, reward, numpy_new_state, done])
                
                # Decide whether to update the main network
                if main_update_counter > main_nn_update_steps:
                    if len(mem) >= replay_mem_min:
                        loss_data[0].append(episode) # append the x-value for the loss-graph
                        train(mem, main_nn, target_nn, optimizer, batch_size, device, loss_data[1])
                    main_update_counter = 0
                
                # Decide whether to update the target network
                if target_update_counter >= target_nn_update_steps:
                    target_nn.load_state_dict(main_nn.state_dict())
                    target_update_counter = 0
                
                # Advance state and add reward to total episode reward
                state = new_state
                episode_reward += reward
                
            # Update episode-parameters add reward to list
            episode += 1
            rewards.append(episode_reward)
            
        print("Training complete!")

    except KeyboardInterrupt:
        print("Training interrupted at episode {0}".format(episode))

In [ ]:
def test_model(model, eps, episodes):
    try:
        
        # Reset environment
        model.to(device)
        reset()

        # Loop through episodes
        for i in range(episodes):
            
            # Reset or update episode-parameters
            done = 0
            state = deque(maxlen = state_size)
            used_steps = 0
            starting_frame = mainProcess.getPixelData(screen_res)
            for i in range(state_size):
                state.append(create_tensor_from_screen_data(starting_frame)) # starting state
            
            # Loop through steps
            while done == 0:

                # Update parameters
                used_steps += 1

                # Decide whether to explore or exploit, and which action to take
                if np.random.uniform(0, 1) < eps:
                    action = np.random.randint(0, num_of_actions - 1)
                else:
                    input_tensor = torch.cat(tuple(state)).unsqueeze(0).to(device)
                    action = np.argmax(model(input_tensor).detach().cpu().numpy())

                # Take action and update state
                frames, done = make_action_realtime(action, screen_res, timestep_interval/1e3, frame_skip) # Take action
                for frame in frames: 
                    state.append(create_tensor_from_screen_data(frame))
                if (used_steps > max_steps):
                    reset()
                    done = 1
            
    except KeyboardInterrupt:
        print("Testing interrupted")

The following function updates the graphs displayed during training. The `rewards`-parameter contains the rewards to be displayed, the `ma_lengths`-parameter is a vector containing the moving averages to apply to the rewards, and the `loss_data` contains the training losses.

In [ ]:
def update_graph(rewards, ma_lengths, loss_data):
    
    # Initialize subplots
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1)
    clear_output(wait = True)
    y = []
    
    # Calculate moving averages
    if len(rewards) > 0:
        for ma_length in ma_lengths:
            if len(rewards) < ma_length: # edge case when there aren't enough samples
                averaged_y = [sum(rewards) / ma_length] * len(rewards)
            else:
                averaged_y = [sum(rewards[:ma_length - 1]) / (ma_length - 1)] * (ma_length - 1) # edge case data
                averaged_y += np.convolve(rewards, np.ones(ma_length)/ma_length, mode = "valid").tolist()
            y.append(averaged_y)
    
    # Raw reward-output
    x = np.arange(len(rewards))
    ax1.plot(x, rewards)
    ax1.set_title("Reward")
    ax1.set(xlabel = "Episode", ylabel = "Reward")
    
    # Moving average of rewards
    for i, line in enumerate(y):
        ax2.plot(x, line, label = "Moving average (N = {})".format(ma_lengths[i]))
    ax2.set_title("Filtered reward")
    ax2.set(xlabel = "Episode", ylabel = "Reward average")
    if len(y) > 0:
        ax2.legend(loc = "upper right", fontsize = 8)
    
    # Loss data
    ax3.plot(loss_data[0], loss_data[1])
    ax3.set_title("Training loss")
    ax3.set(xlabel = "Episode", ylabel = "Training loss (last epoch)")
    
    fig.set_figwidth(10)
    fig.set_figheight(15)
    plt.show()

### Evaluate Model

The following code trains, evaluates and plots data of the Deep Q-learning process. Moreover, an option is given to save and load an existing model. Note that a model can be trained, saved, then trained again by passing a loaded model into `routine` rather than creating a new one.

In [ ]:
# Train model
r = []
net = ActionValueNetwork(screen_dim, screen_res, state_size, num_of_actions)
routine(net, r)

In [ ]:
# Test model
test_model(net, 0.05, 20)

In [ ]:
# Save model
torch.save(net.cpu().state_dict(), 'model.pt')

In [ ]:
# Load model
net = ActionValueNetwork(screen_dim, screen_res, state_size, num_of_actions)
net.load_state_dict(torch.load("model.pt"))